<a href="https://colab.research.google.com/github/denisecammarota/icgpu/blob/main/tpfinal/TimeCorr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Final Project

## GPU checking, setup of Rapids.ai

We check what GPU we have, just in case it's not supported by Rapids.ai, set up Rapids, install from GitHub. 

In [3]:
#checking what gpu google has given us
!nvidia-smi

Tue Jun  1 19:53:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 14.78 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm',

## Auxiliary functions for data processing

Changing numpy for cupy, and pandas for cudf

In [5]:
import cudf
import cupy as cp

In [58]:
import matplotlib.pyplot as plt
import numpy as np

In [15]:
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df2 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    n_size = (df[filt_df2]).shape[0]
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].fillna(0)
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].astype('int')
    df["fecha_apertura"] = df["fecha_apertura"].astype('int')
    b = cp.array(df["fecha_apertura"].values)
    a = cp.array(df["fecha_inicio_sintomas"].values)
    a[filt_df2.values] = b[filt_df2.values] - cp.random.randint(0,9,a[filt_df2.values].shape)
    filt_a = cp.where(a<0)
    a[filt_a] = 0
    df = df.drop(["fecha_inicio_sintomas"],axis=1)
    df["fecha_inicio_sintomas"] = a
    return df

## Data analysis per se, we get data and process 

For this part, don't forget to upload the data from pc to the google drive of the user. First, I authorize colab to get data from my drive for this, so that I don't have to upload to the colab session all the time. We always have to authorize the drive, tho. 

In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
dir = '/content/gdrive/My Drive/Datos_Covid/Covid19Casos.csv'

In [10]:
data = cudf.read_csv(dir,sep=",",quotechar='"',
                   parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) 
data = data[data["clasificacion_resumen"] == "Confirmado"] #filter confirmed cases
df = cudf.DataFrame(data) 

In [11]:
df = df.drop(['clasificacion_resumen'], axis=1) #drop clasificacion_resumen bc previous filter of confirmed cases

In [12]:
filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
inicio_epidemia = (df.loc[filt_df1,"fecha_inicio_sintomas"]).min() #first symptoms of a person registered
ultima_actualizacion_sintomas = (df.loc[filt_df1,"fecha_inicio_sintomas"]).max() #last day symptoms of a person registered
ultima_actualizacion_apertura = (df["fecha_apertura"]).max()
df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
print("primer sintoma de persona confirmada: ",inicio_epidemia)
print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)

primer sintoma de persona confirmada:  2020-01-01T00:00:00.000000000
ultimo sintoma de persona confirmada:  2021-05-28T00:00:00.000000000
ultima apertura de persona confirmada:  2021-05-28T00:00:00.000000000


In [16]:
df = change_dates(df) # we have replaced all bad or undefined dates

In [17]:
df

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,fecha_inicio_sintomas
16,10000015,F,48,Años,Argentina,Entre Ríos,Nogoyá,Entre Ríos,463,14,<NA>,NO,<NA>,NO,<NA>,NO,30,Público,Caso confirmado por laboratorio - No activo (p...,30,2021-04-08,077,2021-05-28,458
27,10000026,F,23,Años,Argentina,Córdoba,Capital,Córdoba,463,14,<NA>,NO,<NA>,NO,<NA>,NO,14,Público,Caso confirmado por laboratorio - No activo (p...,14,2021-04-05,014,2021-05-28,458
51,10000049,F,73,Años,Argentina,CABA,COMUNA 09,CABA,463,14,<NA>,NO,<NA>,NO,<NA>,NO,02,Público,Caso confirmado por laboratorio - No activo (p...,02,2021-04-10,009,2021-05-28,457
58,10000055,F,42,Años,Argentina,CABA,COMUNA 09,CABA,463,14,<NA>,NO,<NA>,NO,<NA>,NO,02,Público,Caso confirmado por laboratorio - No activo (p...,02,2021-04-14,009,2021-05-28,459
60,10000057,M,35,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Público,Caso confirmado por laboratorio - No activo (p...,06,2021-04-06,532,2021-05-28,455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813627,9999973,M,36,Años,Argentina,Buenos Aires,San Isidro,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Privado,Caso confirmado por laboratorio - No activo (p...,06,2021-04-08,756,2021-05-28,455
11813634,9999980,F,17,Años,Argentina,Buenos Aires,San Isidro,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Privado,Caso confirmado por laboratorio - No activo (p...,06,2021-04-09,756,2021-05-28,458
11813640,9999986,M,17,Años,Argentina,Santa Cruz,Güer Aike,Santa Cruz,463,14,<NA>,NO,<NA>,NO,<NA>,NO,78,Público,Caso confirmado por laboratorio - No activo (p...,78,2021-04-09,021,2021-05-28,460
11813647,9999992,F,34,Años,Argentina,Buenos Aires,Merlo,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Público,Caso confirmado por laboratorio - No activo (p...,06,2021-04-09,539,2021-05-28,459


In [18]:
#let's list all the provinces that have cases (all rn tbh, but just to use pandas stuff)
#we'll get those times series then, with cudf grouping
provincias = df["carga_provincia_nombre"].unique()
#print: we get provinces + the sin especificar / unespecified thing
print(provincias)

0            Buenos Aires
1                    CABA
2               Catamarca
3                   Chaco
4                  Chubut
5              Corrientes
6                 Córdoba
7              Entre Ríos
8                 Formosa
9                   Jujuy
10               La Pampa
11               La Rioja
12                Mendoza
13               Misiones
14                Neuquén
15              Río Negro
16                  Salta
17               San Juan
18               San Luis
19             Santa Cruz
20               Santa Fe
21    Santiago del Estero
22       Tierra del Fuego
23                Tucumán
Name: carga_provincia_nombre, dtype: object


## Getting the time series themselves

Now we'll figure out how to get the time series thing working

In [19]:
df = df[["carga_provincia_nombre","fecha_inicio_sintomas"]]

In [20]:
df2 = df.groupby(["carga_provincia_nombre","fecha_inicio_sintomas"]).size().reset_index() 

In [21]:
df2 = df2.rename(columns={0:'casos'})

In [25]:
df3 = df2.sort_values(["carga_provincia_nombre","fecha_inicio_sintomas"],ascending=[True,True])

/usr/local/lib/python3.7/site-packages/cudf/core/frame.py:2582: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


In [47]:
rows = provincias.shape[0]
cols = df2["fecha_inicio_sintomas"].max()
time_series = cp.zeros(shape=(rows,cols))

In [48]:
j = 0
for i in provincias.to_pandas():
  df_tmp = df3[df3["carga_provincia_nombre"]==i]
  tmp_dates = cp.array(df_tmp["fecha_inicio_sintomas"].values)
  tmp_cases = cp.array(df_tmp["casos"].values)
  time_series[j,tmp_dates]= tmp_cases
  j = j + 1

## Correlation finding techniques

All adapted from: https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9

### Pearson R programmed in CuPy

In [79]:
#no existe en cupy, la hago yo a esta funcion 
def pearsonr(a,b):
  m_a = cp.mean(a)
  m_b = cp.mean(b)
  r_a = a - m_a
  r_b = b - m_b
  r_a_2 = cp.sum((r_a)**2) 
  r_b_2 = cp.sum((r_b)**2)
  r = cp.sum((r_a*r_b)/cp.sqrt(r_a_2*r_b_2))
  return r

In [82]:
#por ejemplo, correlaciones para caba y baires
caba = time_series[1,:]
baires = time_series[0,:]
pearsonr(caba,baires)

array(0.97326164)

In [106]:
#ahora, tratemos de hacer lo mismo para baires y el resto del país
baires = time_series[0,:]
i = 1
provs = provincias.to_pandas()
print('Coeficientes de Pearson')
while i < rows:
  prov_tmp = time_series[i,:]
  r = pearsonr(baires,prov_tmp)
  print('Pearson entre '+provs[i]+' y Baires: ',cp.around(r,3))
  i += 1

Coeficientes de Pearson
Pearson entre CABA y Baires:  0.973
Pearson entre Catamarca y Baires:  0.741
Pearson entre Chaco y Baires:  0.762
Pearson entre Chubut y Baires:  0.431
Pearson entre Corrientes y Baires:  0.635
Pearson entre Córdoba y Baires:  0.8
Pearson entre Entre Ríos y Baires:  0.813
Pearson entre Formosa y Baires:  0.483
Pearson entre Jujuy y Baires:  0.567
Pearson entre La Pampa y Baires:  0.709
Pearson entre La Rioja y Baires:  0.77
Pearson entre Mendoza y Baires:  0.861
Pearson entre Misiones y Baires:  0.571
Pearson entre Neuquén y Baires:  0.633
Pearson entre Río Negro y Baires:  0.69
Pearson entre Salta y Baires:  0.777
Pearson entre San Juan y Baires:  0.704
Pearson entre San Luis y Baires:  0.691
Pearson entre Santa Cruz y Baires:  0.555
Pearson entre Santa Fe y Baires:  0.745
Pearson entre Santiago del Estero y Baires:  0.707
Pearson entre Tierra del Fuego y Baires:  0.42
Pearson entre Tucumán y Baires:  0.664


'Buenos Aires'

### Time Lagged Cross Correlation